In [ ]:
# %pip install progressbar xgboost matplotlib boto3 openpyxl tqdm hydroeval hydrotools

In [1]:
# hydrological packages
from hydrotools.nwm_client import utils 

# my packages
from evaluation_metric import MAPE, RMSE, KGE, PBias

# basic packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# system packages
from progressbar import ProgressBar
from datetime import datetime, date
import pickle
import warnings
warnings.filterwarnings("ignore")
import platform
import time

# data analysi packages
from scipy import optimize

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# deep learning packages
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


In [2]:
if platform.system() == 'Windows':
    onedrive_path = 'E:/OneDrive/OneDrive - The University of Alabama/10.material/01.data/usgs_data/'
    box_path = 'C:/Users/snaserneisary/Box/NWM-ML/'

elif platform.system() == 'Darwin':
    onedrive_path = '/Users/savalan/Library/CloudStorage/OneDrive-TheUniversityofAlabama/02.projects/03.ciroh/04.data/'
    box_path = '/Users/savalan/Library/CloudStorage/Box-Box/NWM-ML/Data/NWM/ut/'
    
elif platform.system() == 'Linux':
    path_01 = '/home/snaserneisary/01.projects/01.ciroh_p8/NWM-ML/Savalan/'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


In [4]:
raw_training_data = pd.read_csv(path_01 + '03.output/raw_training_data.csv')
raw_training_data.pop('Unnamed: 0')
raw_training_data['station_id'] = raw_training_data['station_id'].astype('str')
raw_training_data.head()

,station_id,Lat,Long,Drainage_area_mi2,Mean_Basin_Elev_ft,Perc_Forest,Perc_Develop,Perc_Imperv,Perc_Herbace,Perc_Slop_30,...,datetime,flow_cfs,s1,s2,temperature_F,precipitation_in,storage,swe,NWM_flow,DOY
0,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2012-03-13,45.356945,0.515038,0.857167,35.096,0.00000,0.0,7.70,60.0,73
1,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2012-03-14,49.750000,0.515038,0.857167,35.258,0.00000,0.0,7.45,62.0,74
2,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2012-03-15,52.483334,0.515038,0.857167,36.860,0.00000,0.0,7.35,65.0,75
3,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2012-03-16,60.296875,0.515038,0.857167,38.120,0.00000,0.0,7.25,63.0,76
4,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2012-03-17,68.876045,0.515038,0.857167,38.102,0.04698,0.0,6.85,65.0,77


In [5]:
Training_DF = raw_training_data.copy()

### Editing the features based on the feature importance should be in the next cell!!!!!!!!!!!!!!!

In [6]:
# Editing the features based on the feature importance should be done here!!!!!!!!!!!!!!!

Training_DF.drop(['precipitation_in', 'temperature_F', 'Mean_Ann_Precip_in', 'Perc_Herbace', 'Perc_Forest',
                        'Mean_Basin_Elev_ft'], axis=1, inplace=True)



### Remove headwater stations!!!!!!!

In [7]:
headwater_stations = ['10011500', '10109000', '10113500', '10128500', '10131000', '10146400', '10150500', '10154200',
'10172700', '10172800', '10172952']
Training_DF = Training_DF[~raw_training_data['station_id'].isin(headwater_stations)]

In [8]:
Training_DF.datetime = pd.to_datetime(Training_DF.datetime)
Training_DF.head()

,station_id,Lat,Long,Drainage_area_mi2,Perc_Develop,Perc_Imperv,Perc_Slop_30,datetime,flow_cfs,s1,s2,storage,swe,NWM_flow,DOY
789,10126000,41.576321,-112.100782,7040.0,4.28,0.55,1.94,2012-03-01,1488.4375,0.515038,0.857167,85.588235,12.778571,1542.0,61
790,10126000,41.576321,-112.100782,7040.0,4.28,0.55,1.94,2012-03-02,1282.1875,0.515038,0.857167,86.712418,13.085714,1542.0,62
791,10126000,41.576321,-112.100782,7040.0,4.28,0.55,1.94,2012-03-03,1269.4791,0.515038,0.857167,87.359477,13.178571,1542.0,63
792,10126000,41.576321,-112.100782,7040.0,4.28,0.55,1.94,2012-03-04,1551.5625,0.515038,0.857167,88.163399,13.221429,1541.0,64
793,10126000,41.576321,-112.100782,7040.0,4.28,0.55,1.94,2012-03-05,1490.5209,0.515038,0.857167,89.026144,13.221429,1541.0,65


In [9]:
x_train_temp = Training_DF[Training_DF.datetime < '01-01-2015']
x_train_temp.pop('station_id')
x_train_temp.pop('datetime')
y_train_temp = x_train_temp['flow_cfs']
x_train_temp.pop('flow_cfs')
x_train_temp.head()

,Lat,Long,Drainage_area_mi2,Perc_Develop,Perc_Imperv,Perc_Slop_30,s1,s2,storage,swe,NWM_flow,DOY
789,41.576321,-112.100782,7040.0,4.28,0.55,1.94,0.515038,0.857167,85.588235,12.778571,1542.0,61
790,41.576321,-112.100782,7040.0,4.28,0.55,1.94,0.515038,0.857167,86.712418,13.085714,1542.0,62
791,41.576321,-112.100782,7040.0,4.28,0.55,1.94,0.515038,0.857167,87.359477,13.178571,1542.0,63
792,41.576321,-112.100782,7040.0,4.28,0.55,1.94,0.515038,0.857167,88.163399,13.221429,1541.0,64
793,41.576321,-112.100782,7040.0,4.28,0.55,1.94,0.515038,0.857167,89.026144,13.221429,1541.0,65


In [10]:
# Scale the train inputs of the NN model
# First we need to convert it from pandas dataframe to a numpy array 
y_train = y_train_temp.to_numpy()
x_train = x_train_temp.to_numpy()
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
y_scaled_train = scaler.fit_transform(y_train.reshape(-1, 1))
y_scaled_train.shape

(8174, 1)

In [11]:
# Determining the test dataset. 
x_test_temp = Training_DF[Training_DF.datetime >= '01-01-2015']
x_test_temp.head()

,station_id,Lat,Long,Drainage_area_mi2,Perc_Develop,Perc_Imperv,Perc_Slop_30,datetime,flow_cfs,s1,s2,storage,swe,NWM_flow,DOY
874,10126000,41.576321,-112.100782,7040.0,4.28,0.55,1.94,2015-03-01,723.29170,0.515038,0.857167,79.712418,10.171429,1658.0,60
875,10126000,41.576321,-112.100782,7040.0,4.28,0.55,1.94,2015-03-02,742.03125,0.515038,0.857167,80.156863,10.185714,1668.0,61
876,10126000,41.576321,-112.100782,7040.0,4.28,0.55,1.94,2015-03-03,729.18750,0.515038,0.857167,80.620915,10.335714,1682.0,62
877,10126000,41.576321,-112.100782,7040.0,4.28,0.55,1.94,2015-03-04,792.37500,0.515038,0.857167,80.967320,10.564286,1701.0,63
878,10126000,41.576321,-112.100782,7040.0,4.28,0.55,1.94,2015-03-05,848.16670,0.515038,0.857167,81.320261,10.678571,1720.0,64


In [12]:
# Scale the test inputs of the NN model
# First we need to convert it from pandas dataframe to a numpy array 
x_test_temp_1 = x_test_temp.copy()
station_index_list = x_test_temp_1['station_id']
x_test_temp_1.pop('station_id')
x_test_temp_1.pop('datetime')
y_test_temp_1 = x_test_temp_1['flow_cfs']
x_test_temp_1.pop('flow_cfs')
x_test_1_np = x_test_temp_1.reset_index(drop=True).to_numpy()
y_test_1_np = y_test_temp_1.reset_index(drop=True).to_numpy()
x_test_1_scaled = scaler.fit_transform(x_test_1_np)
y_scaled_test_1 = scaler.fit_transform(y_test_1_np.reshape(-1, 1))

In [13]:
# MLP

n_targets = 1
tries = 10
cri_temp_nse = np.zeros([3, n_targets, tries])
cri_temp_rmse = np.zeros([3, n_targets, tries])
cri_temp_r2 = np.zeros([3, n_targets, tries])
cri_temp_kge = np.zeros([3, n_targets, tries])
cri_temp_lognse = np.zeros([3, n_targets, tries])
print('finish')

finish


In [14]:
# Reshape input for MLP model
x_train_scaled_test = torch.Tensor(x_train_scaled)
y_train_scaled_test = torch.Tensor(y_scaled_train)
print('test shape', x_train_scaled_test.shape)
print('train shape', y_train_scaled_test.shape)

test shape torch.Size([8174, 12])
train shape torch.Size([8174, 1])


In [17]:
x_train_scaled_test[:5, :]

tensor([[1.0000, 0.0000, 1.0000, 0.2584, 0.1224, 0.0000, 0.7574, 0.9286, 0.8422,
         0.6230, 0.2866, 0.1644],
        [1.0000, 0.0000, 1.0000, 0.2584, 0.1224, 0.0000, 0.7574, 0.9286, 0.8533,
         0.6380, 0.2866, 0.1671],
        [1.0000, 0.0000, 1.0000, 0.2584, 0.1224, 0.0000, 0.7574, 0.9286, 0.8597,
         0.6425, 0.2866, 0.1699],
        [1.0000, 0.0000, 1.0000, 0.2584, 0.1224, 0.0000, 0.7574, 0.9286, 0.8676,
         0.6446, 0.2864, 0.1726],
        [1.0000, 0.0000, 1.0000, 0.2584, 0.1224, 0.0000, 0.7574, 0.9286, 0.8761,
         0.6446, 0.2864, 0.1753]])

In [15]:
# Take data to the proper device (GPU or CPU)
x_train_scaled_test = x_train_scaled_test.to(device)
y_train_scaled_test = y_train_scaled_test.to(device)

In [16]:
# Hyperparameters
epochs = 100
batch_size = 100
learning_rate = 0.001
decay = 1e-2
validation_split = 0.2
neurons = 150
LD1=128
LD2=128
LD3=64
LD4=64
LD5=32
LD6=16
LD7=5

In [17]:
start_time = time.time()

# Create PyTorch datasets and dataloaders
train_dataset = TensorDataset(x_train_scaled_test, y_train_scaled_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

# Build the model
model = nn.Sequential(
    nn.Linear(x_test_1_scaled.shape[1], LD1),
    nn.ReLU(),
    nn.Linear(LD1, LD2),
    nn.ReLU(),
    nn.Linear(LD2, LD3),
    nn.ReLU(),
    nn.Linear(LD3, LD4),
    nn.ReLU(),
    nn.Linear(LD4, LD5),
    nn.ReLU(),
    nn.Linear(LD5, LD6),
    nn.ReLU(),
    nn.Linear(LD6, 1)
).to(device)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    total_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")
print('finish')
print("Run Time:" + " %s seconds " % (time.time() - start_time))
#save model
#torch.save(model.state_dict(), f"./mlp_models/mlp_model.pkl")

Epoch 1/100, Loss: 0.018824823003541963
Epoch 2/100, Loss: 0.005690550160269174
Epoch 3/100, Loss: 0.004863848974465083
Epoch 4/100, Loss: 0.004205533485219348
Epoch 5/100, Loss: 0.003589668937853808
Epoch 6/100, Loss: 0.003086815867036199
Epoch 7/100, Loss: 0.0027412831850819113
Epoch 8/100, Loss: 0.0025072536676949623
Epoch 9/100, Loss: 0.002227033212076959
Epoch 10/100, Loss: 0.0022975615309672034
Epoch 11/100, Loss: 0.0020639144158183893
Epoch 12/100, Loss: 0.002052941399841341
Epoch 13/100, Loss: 0.0020370934198359553
Epoch 14/100, Loss: 0.0021460719898663584
Epoch 15/100, Loss: 0.0023476182867320527
Epoch 16/100, Loss: 0.0026975349996862905
Epoch 17/100, Loss: 0.0030458004098585572
Epoch 18/100, Loss: 0.002958192373847762
Epoch 19/100, Loss: 0.003246909743943323
Epoch 20/100, Loss: 0.0035852983656673686
Epoch 21/100, Loss: 0.0032166865154378194
Epoch 22/100, Loss: 0.002220267493967407
Epoch 23/100, Loss: 0.001892213825452173
Epoch 24/100, Loss: 0.0016720968123880128
Epoch 25/100,

In [18]:
# # Build and load the model
device = torch.device('cpu') # for some reason had to change to cpu
models = nn.Sequential(
    nn.Linear(x_test_1_scaled.shape[1], LD1),
    nn.ReLU(),
    nn.Linear(LD1, LD2),
    nn.ReLU(),
    nn.Linear(LD2, LD3),
    nn.ReLU(),
    nn.Linear(LD3, LD4),
    nn.ReLU(),
    nn.Linear(LD4, LD5),
    nn.ReLU(),
    nn.Linear(LD5, LD6),
    nn.ReLU(),
    nn.Linear(LD6, 1)
).to(device)

models.load_state_dict(torch.load(f"{path_01}03.output/mlp/mlp_model.pkl", map_location=torch.device('cpu')))

EvalDF = pd.DataFrame(columns = cols)
SupplyEvalDF = pd.DataFrame(columns = supcols)

SitesDict = {}

for station_number in station_index_list.drop_duplicates():
    index = station_index_list == station_number # Finind the rows that have this station number.
    X_test_scaled = torch.Tensor(x_test_1_scaled[index])
    
    # Evaluation
    models.eval()
    with torch.no_grad():
        yhat_test= models(X_test_scaled)

    # Invert scaling for actual
    inv_yhat_test = scaler.inverse_transform(yhat_test.numpy())
    inv_yhat_test[inv_yhat_test<0] = 0 # THIS IS NOT CORRECT !!!!!!!!!!!!!!!
    nwm_test = pd.DataFrame(inv_yhat_test, columns=['MLP_flow'])
    Dfs = [nwm_test.reset_index(drop=True), x_test_temp[index].reset_index(drop=True)]

    #get reach id for model eval
    nhdreach = utils.crosswalk(usgs_site_codes=station_number)
    nhdreach = nhdreach['nwm_feature_id'].iloc[0]
    
    #save predictions
    mod = 'MLP'
    save_path = f"./Predictions/Hindcast/{mod}/{mod}_{nhdreach}.csv"
    #nwm_test.to_csv(save_path)
    
    # merge
    Eval_DF_mine = pd.concat(Dfs, axis=1)
    SitesDict[nhdreach] = Eval_DF_mine

    
    prediction_columns = ['NWM_flow', 'MLP_flow']
    observation_column = 'flow_cfs'
    
    #Get RMSE from the model
    rmse = RMSE(Eval_DF_mine, prediction_columns, observation_column)

    #Get Mean Absolute Percentage Error from the model
    mape = MAPE(Eval_DF_mine, prediction_columns, observation_column)

    #Get Percent Bias from the model
    pbias = PBias(Eval_DF_mine, prediction_columns, observation_column)

    #Get Kling-Gutz Efficiency from the model
    kge = KGE(Eval_DF_mine, prediction_columns, observation_column)
    
    #Get Volumetric values
    Eval_DF_mine.set_index('datetime', inplace = True, drop =True)
    flowcols = [f"{mod}_flow", 'flow_cfs', 'NWM_flow']
    SupplyEval = Eval_DF_mine[flowcols].copy()
    SupplyEval = SupplyEval*cfsday_AFday
    #set up cumulative monthly values
    SupplyEval['Year'] = SupplyEval.index.year

    for site in flowcols:
        SupplyEval[site] = SupplyEval.groupby(['Year'])[site].cumsum()  

    EOY_mod_vol_af = SupplyEval[f"{mod}_flow"].iloc[-1]
    EOY_obs_vol_af = SupplyEval["flow_cfs"].iloc[-1]
    EOY_nwm_vol_af = SupplyEval[f"NWM_flow"].iloc[-1]
    NWM_vol_diff_af = EOY_nwm_vol_af - EOY_obs_vol_af
    Mod_vol_diff_af = EOY_mod_vol_af - EOY_obs_vol_af
    NWM_Perc_diff = (NWM_vol_diff_af/EOY_obs_vol_af)*100
    Mod_Perc_diff = (Mod_vol_diff_af/EOY_obs_vol_af)*100
    
     #Get Performance Metrics from the model
    Srmse = RMSE(SupplyEval, prediction_columns, observation_column)
    Smape = MAPE(SupplyEval, prediction_columns, observation_column)
    Spbias = PBias(SupplyEval, prediction_columns, observation_column)
    Skge = KGE(SupplyEval, prediction_columns, observation_column)
    
    
    #save model performance
    sitestats = [station_number, nhdreach, rmse[0], rmse[1],  pbias[0], pbias[1], kge[0], kge[1], mape[0],mape[1]]
    EvalDF.loc[len(EvalDF)] = sitestats
    
    Supplystats = [station_number, nhdreach, Srmse[0], Srmse[1],  Spbias[0], Spbias[1], Skge[0], Skge[1], Smape[0],  
                 Smape[1],EOY_obs_vol_af, EOY_nwm_vol_af,EOY_mod_vol_af,NWM_vol_diff_af,Mod_vol_diff_af, NWM_Perc_diff, Mod_Perc_diff ]
    SupplyEvalDF.loc[len(SupplyEvalDF)] = Supplystats
    
    
    
    
    #put prediction DF into dictionary
    # Eval_DF_mine.reset_index(inplace = True)
    # Eval_DF_mine.sort_values(by=['datetime'], inplace=True)
    # Eval_DF_mine.set_index('datetime', inplace = True)
    
    
    
#save model results
#EvalDF.to_csv(f"./Predictions/Hindcast/{mod}/{mod}_Performance.csv")   
#SupplyEvalDF.to_csv(f"./Predictions/Hindcast/{mod}/{mod}_Supply_Performance.csv")

print('finish')


NameError: name 'cols' is not defined

In [36]:
mod = 'MLP'

In [31]:
from evalaution_table import evtab
# # Build and load the model
device = torch.device('cpu') # for some reason had to change to cpu
# models = nn.Sequential(
#     nn.Linear(x_test_1_scaled.shape[1], LD1),
#     nn.ReLU(),
#     nn.Linear(LD1, LD2),
#     nn.ReLU(),
#     nn.Linear(LD2, LD3),
#     nn.ReLU(),
#     nn.Linear(LD3, LD4),
#     nn.ReLU(),
#     nn.Linear(LD4, LD5),
#     nn.ReLU(),
#     nn.Linear(LD5, LD6),
#     nn.ReLU(),
#     nn.Linear(LD6, 1)
# ).to(device)

#model.load_state_dict(torch.load(f"{path_01}03.output/mlp/mlp_model.pkl", map_location=torch.device('cpu')))
mod = 'MLP'
EvalDF = np.zeros([len(station_index_list.drop_duplicates()), 10])
SupplyEvalDF = np.zeros([len(station_index_list.drop_duplicates()), 17])

SitesDict = {}

for station_index, station_number in enumerate(station_index_list.drop_duplicates()):
    index = station_index_list == station_number # Finind the rows that have this station number.
    X_test_scaled = torch.Tensor(x_test_1_scaled[index])
    
    # Evaluation
    model.eval()
    with torch.no_grad():
        yhat_test= model(X_test_scaled)

    # Invert scaling for actual
    inv_yhat_test = scaler.inverse_transform(yhat_test.numpy())
    inv_yhat_test[inv_yhat_test<0] = 0 # THIS IS NOT CORRECT !!!!!!!!!!!!!!!
    nwm_test = pd.DataFrame(inv_yhat_test, columns=['MLP_flow'])
    Dfs = [nwm_test.reset_index(drop=True), x_test_temp[index].reset_index(drop=True)]

    #get reach id for model eval
    nhdreach = utils.crosswalk(usgs_site_codes=station_number)
    nhdreach = nhdreach['nwm_feature_id'].iloc[0]
    
    #save predictions
    #save_path = f"./Predictions/Hindcast/{mod}/{mod}_{nhdreach}.csv"
    #nwm_test.to_csv(save_path)
    
    # merge
    Eval_DF_mine = pd.concat(Dfs, axis=1)
    SitesDict[nhdreach] = Eval_DF_mine
    prediction_columns = ['NWM_flow', f"{mod}_flow"]
    observation_column = 'flow_cfs'
    result = evtab(Eval_DF_mine, prediction_columns, nhdreach, observation_column, mod)
    
    #save model performance
    EvalDF[station_index, :] = result[0]
    SupplyEvalDF[station_index, :] = result[1]
    
    

print('finish')


finish


In [32]:
nwm_test

,MLP_flow
0,123.224350
1,122.234627
2,121.262222
3,120.307045
4,119.428665
...,...
2093,201.519714
2094,200.848557
2095,200.142426
2096,199.304352


In [28]:
#put the model scores into a dataframe for comparison
mod = 'MLP'
#Evaluation columns for prediction time series
cols = ['USGSid', 'NHDPlusid', 'NWM_rmse', f"{mod}_rmse", 'NWM_pbias', f"{mod}_pbias", 
        'NWM_kge', f"{mod}__kge", 'NWM_mape',  f"{mod}_mape"]

#Evaluation columns for accumulated supply time series
supcols = ['USGSid', 'NHDPlusid', 'NWM_rmse', f"{mod}_rmse", 'NWM_pbias', f"{mod}_pbias", 
        'NWM_kge', f"{mod}__kge", 'NWM_mape',  f"{mod}_mape", 'Obs_vol', 'NWM_vol', f"{mod}_vol",
        'NWM_vol_err', f"{mod}_vol_err", 'NWM_vol_Perc_diff', f"{mod}_vol_Perc_diff"]
    
#save model results
EvalDF = pd.DataFrame(EvalDF, columns=cols)
SupplyEvalDF = pd.DataFrame(SupplyEvalDF, columns=supcols)

#EvalDF.to_csv(f"./Predictions/Hindcast/{mod}/{mod}_Performance.csv")   
#SupplyEvalDF.to_csv(f"./Predictions/Hindcast/{mod}/{mod}_Supply_Performance.csv")


In [30]:
print("Model Performance for Daily cfs")
display(EvalDF)   
print("Model Performance for Daily Accumulated Supply (Acre-Feet)")
display(SupplyEvalDF )

Model Performance for Daily cfs


,USGSid,NHDPlusid,NWM_rmse,MLP_rmse,NWM_pbias,MLP_pbias,NWM_kge,MLP__kge,NWM_mape,MLP_mape
0,10126000.0,4605050.0,1861.439475,1158.135184,-53.15,-7.21,0.14,0.51,369.37,97.47
1,10130500.0,10092262.0,305.608055,151.860726,-114.84,29.37,-0.80,0.28,197.11,81.43
2,10134500.0,10277268.0,128.603024,56.466362,-235.17,83.77,-1.48,-0.11,1014.93,93.31
3,10136500.0,10274616.0,776.608631,372.127163,-191.07,33.31,-0.99,0.13,436.46,71.38
4,10137500.0,10274270.0,107.073089,146.890962,33.05,75.11,0.48,-0.01,38.66,89.17
5,10141000.0,10273232.0,1317.964422,546.581092,-412.87,77.48,-3.20,-0.47,1670.95,95.81
6,10155000.0,10373622.0,317.555275,358.992849,21.78,34.02,0.47,0.50,123.39,88.12
7,10164500.0,10329013.0,110.840940,69.677646,-123.23,100.00,-0.95,NaN,130.71,100.00
8,10171000.0,10390290.0,3558.880086,72.724457,-2822.39,-0.55,-29.81,-0.35,3342.15,58.51


Model Performance for Daily Accumulated Supply (Acre-Feet)


,USGSid,NHDPlusid,NWM_rmse,MLP_rmse,NWM_pbias,MLP_pbias,NWM_kge,MLP__kge,NWM_mape,MLP_mape,Obs_vol,NWM_vol,MLP_vol,NWM_vol_err,MLP_vol_err,NWM_vol_Perc_diff,MLP_vol_Perc_diff
0,10126000.0,4605050.0,4.525278e+05,287897.925341,-55.72,2.75,0.14,0.48,74.17,49.06,655183.188796,1149487.593,895593.250000,4.943044e+05,240410.061204,75.445221,36.693564
1,10130500.0,10092262.0,8.536463e+04,26670.926644,-120.90,31.02,-0.72,0.57,163.42,39.86,77596.594340,157380.795,62627.257812,7.978420e+04,-14969.336527,102.819204,-19.291229
2,10134500.0,10277268.0,4.845263e+04,19572.717901,-213.52,85.92,-1.77,-0.17,521.08,91.11,37262.276058,96970.683,6460.203125,5.970841e+04,-30802.072933,160.238217,-82.662886
3,10136500.0,10274616.0,2.959131e+05,118146.383561,-155.58,39.76,-0.86,0.20,243.69,44.79,172949.053345,490788.534,137462.359375,3.178395e+05,-35486.693970,183.776363,-20.518582
4,10137500.0,10274270.0,2.529990e+04,53329.543887,38.01,70.54,0.60,-0.17,45.32,61.46,54744.014329,39350.652,15876.970703,-1.539336e+04,-38867.043626,-28.118804,-70.997796
5,10141000.0,10273232.0,5.382548e+05,177624.696931,-363.73,89.40,-2.97,-0.40,977.87,74.44,120033.770763,685973.241,34005.328125,5.659395e+05,-86028.442638,471.483539,-71.670199
6,10155000.0,10373622.0,6.066519e+04,71806.335509,31.76,28.15,0.59,0.43,38.25,48.32,181010.801007,126493.587,108523.046875,-5.451721e+04,-72487.754132,-30.118210,-40.046093
7,10164500.0,10329013.0,3.034985e+04,24051.464152,-140.03,100.00,-0.63,NaN,183.25,100.00,19609.368842,59842.974,0.000000,4.023361e+04,-19609.368842,205.175421,-100.000000
8,10171000.0,10390290.0,1.481821e+06,11578.558096,-2948.53,18.06,-38.73,0.81,3366.51,27.11,81024.872027,1912801.800,66664.085938,1.831777e+06,-14360.786089,2260.758804,-17.723923


In [19]:
print("Model Performance for Daily cfs")
display(EvalDF)   
print("Model Performance for Daily Accumulated Supply (Acre-Feet)")
display(SupplyEvalDF )

Model Performance for Daily cfs


,USGSid,NHDPlusid,NWM_rmse,MLP_rmse,NWM_pbias,MLP_pbias,NWM_kge,MLP__kge,NWM_mape,MLP_mape
0,10126000,4605050,1861.439475,692.033427,-53.15,-12.82,0.14,0.78,369.37,52.63
1,10130500,10092262,305.608055,120.717862,-114.84,20.07,-0.80,0.24,197.11,70.01
2,10134500,10277268,128.603024,47.540663,-235.17,39.93,-1.48,0.09,1014.93,170.26
3,10136500,10274616,776.608631,216.496869,-191.07,-12.84,-0.99,0.81,436.46,69.35
4,10137500,10274270,107.073089,137.583271,33.05,38.44,0.48,0.08,38.66,63.44
5,10141000,10273232,1317.964422,229.866345,-412.87,-8.40,-3.20,0.85,1670.95,96.50
6,10155000,10373622,317.555275,226.989005,21.78,2.64,0.47,0.83,123.39,64.87
7,10164500,10329013,110.840940,54.321582,-123.23,40.58,-0.95,0.06,130.71,120.70
8,10171000,10390290,3558.880086,42.771226,-2822.39,-1.92,-29.81,-0.09,3342.15,39.21


Model Performance for Daily Accumulated Supply (Acre-Feet)


,USGSid,NHDPlusid,NWM_rmse,MLP_rmse,NWM_pbias,MLP_pbias,NWM_kge,MLP__kge,NWM_mape,MLP_mape,Obs_vol,NWM_vol,MLP_vol,NWM_vol_err,MLP_vol_err,NWM_vol_Perc_diff,MLP_vol_Perc_diff
0,10126000,4605050,4.525278e+05,135601.368137,-55.72,-11.09,0.14,0.74,74.17,17.65,655183.188796,1149487.593,767235.562500,4.943044e+05,112052.373704,75.445221,17.102449
1,10130500,10092262,8.536463e+04,18659.155726,-120.90,14.88,-0.72,0.61,163.42,40.42,77596.594340,157380.795,72287.570312,7.978420e+04,-5309.024027,102.819204,-6.841826
2,10134500,10277268,4.845263e+04,9943.377293,-213.52,33.60,-1.77,0.38,521.08,110.20,37262.276058,96970.683,21517.763672,5.970841e+04,-15744.512386,160.238217,-42.253222
3,10136500,10274616,2.959131e+05,21430.928363,-155.58,-10.09,-0.86,0.89,243.69,29.58,172949.053345,490788.534,206140.578125,3.178395e+05,33191.524780,183.776363,19.191504
4,10137500,10274270,2.529990e+04,41066.102563,38.01,46.02,0.60,0.13,45.32,46.84,54744.014329,39350.652,47727.761719,-1.539336e+04,-7016.252611,-28.118804,-12.816474
5,10141000,10273232,5.382548e+05,25166.789448,-363.73,-9.14,-2.97,0.87,977.87,37.30,120033.770763,685973.241,75881.437500,5.659395e+05,-44152.333263,471.483539,-36.783259
6,10155000,10373622,6.066519e+04,27428.799109,31.76,2.28,0.59,0.83,38.25,33.34,181010.801007,126493.587,167993.812500,-5.451721e+04,-13016.988507,-30.118210,-7.191277
7,10164500,10329013,3.034985e+04,14423.525165,-140.03,42.68,-0.63,0.13,183.25,78.26,19609.368842,59842.974,16443.640625,4.023361e+04,-3165.728217,205.175421,-16.143958
8,10171000,10390290,1.481821e+06,7273.459034,-2948.53,2.53,-38.73,0.94,3366.51,22.11,81024.872027,1912801.800,65987.796875,1.831777e+06,-15037.075152,2260.758804,-18.558592


In [ ]:
#save model results
EvalDF.to_csv(f"./Predictions/Hindcast/{mod}/{mod}_Performance.csv")

In [ ]:
import importlib
importlib.reload(FigureGenerator)

In [ ]:
import FigureGenerator

model = 'MLP'
plotname = 'MLP_TS_plot'
freq = 'D'
supply = True
title = 'Observed and Modeled flows for NHDPlus Reaches \n with Upstream Reservoirs in the Great Salt Lake Basin'
FigureGenerator.TS_plot(SitesDict, model, plotname, title, freq, supply)

In [ ]:
plotname = 'MLP_ParityPlot'
FigureGenerator.Parity_plot(SitesDict, model, plotname)

In [ ]:
import AWS_transfer
model = 'MLP'
state = 'ut'
AWS_transfer.Predictions2AWS(model, state)

In [ ]:
reach = 10273232
variables =['NWM_flow', 'Obs_flow']
colors = ['blue', 'green']
model = 'MLP'
plotname = 'NWMFlow'
units = 'cfs'
y_lab = f"Flow ({units})"
title = f"Daily NWM Estimates \n Reach: {str(reach)}"


FigureGenerator.Var_TS_plot(SitesDict, reach, variables, colors, model,y_lab, plotname, title, units, supply = False)